# Lab 2, Question 6

(6) Exercise: using an SVM model and the Landcare NZ 2024 landcover database, produce a landcover map of Great Barrier (Aotea) Island for 2025 (based off the Austral summer of 24/25).

Your map should be presented at a publication quality level with all the usual map components (scale, legend, north arrow, data attribution).

You will need to provide performance statistics of the model within your figure.

*   Here you can access the landcover database: https://lris.scinfo.org.nz/layer/104400-lcdb-v50-land-cover-database-version-50-mainland-new-zealand/. You will need to explore for yourself how to extract this data and then upload it to colab, then how to plug it into the SVM algorithim. I have provided some starter code below.

An intial workflow to get the data into the state you need it in to then use it as training data might look like:
- Download the ZIP manually from their browser, having set your area of interest and used the 'Export' tool top right.
- Upload it to Colab.
- Unzip it and load with GeoPandas.
(25 pts)


_______________________________________________________________________________


### Setup

In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    EE_PROJECT_ID = userdata.get('EE_PROJECT_ID') 
else:
    from dotenv import load_dotenv
    import os
    load_dotenv()  # take environment variables
    EE_PROJECT_ID = os.getenv('EE_PROJECT_ID')

# Set up GEE API
import ee
ee.Authenticate()
ee.Initialize(project=EE_PROJECT_ID) #<- Remember to change this to your own project's name!

In [ ]:
import tempfile
import urllib.request

from IPython.display import Image

import geemap
import geopandas as gpd
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from sklearn.metrics import confusion_matrix, classification_report

# A. Read Data

### A.1. Load LCDB v5.0 - Land Cover Database version 5.0, Mainland, New Zealand for 2024

In [ ]:
seed = 42  # Random seed for reproducibility

# # Coordinates for Mount Hobson
# point = ee.Geometry.Point([175.3785, -36.1830])
aoi = ee.Geometry.Rectangle([175.27, -36.01, 175.57, -36.36])

# bands = ['B2', 'B3', 'B4', 'B8'] # Sentinel-2 bands: Blue, Green, Red, NIR
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

# lcdb = geemap.geopandas_to_ee(df_2018[[class_property, 'geometry']])

# # rasterizes the data
# lcdb_image = lcdb.reduceToImage(
#     properties=[class_property],
#     reducer=ee.Reducer.first()
# ).clip(aoi).rename(class_property)

def mask_s2_clouds(image):
    """Masks clouds in your S2 images via QA60 band"""
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
    qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).select(bands)

s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterBounds(aoi)
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)))

s2_2018 = (s2
           .filterDate('2018-01-01', '2018-12-31')
           .map(mask_s2_clouds)
           .median()
           .clip(aoi))

# # `lcdb` only covers land. Need to mark unlabled areas in `s2_2018`.
# s2_2018_lc = s2_2018.updateMask(lcdb_image)
# training_image = s2_2018_lc.addBands(lcdb_image)

# # Combine the bands of interest with the land cover data
# # The `class_property` band will be used as our label
# training_image = s2_2018_lc.addBands(lcdb_image)

### ESA BEGIN

In [ ]:
s2_esa_2020 = (s2
           .filterDate('2020-01-01', '2020-12-31')
           .map(mask_s2_clouds)
           .median()
           .clip(aoi))

# Define valid WorldCover classes (10 to 100, spaced by 10)
valid_classes = ee.List.sequence(10, 100, 10) # 100 is included
# list(range(10, 100, 10)) # 100 in not included

# Need to change this to sequential list to avoid the algs thinking there are 99 classes rather than 9
remap_to = ee.List.sequence(1, 10)

# Load and mask WorldCover map to valid classes only
landcover = ee.Image('ESA/WorldCover/v100/2020').select('Map').clip(aoi)
landcover_masked = landcover.updateMask(
    landcover.remap(valid_classes, ee.List.repeat(1, 10)))

# Remap labels: create a new band with remapped values (10 → 1, ..., 100 → 10)
landcover_remapped = landcover.remap(valid_classes, remap_to).rename('Map_remapped')

# Add landcover original and remapped as bands to Sentinel-2 image
training_data = s2_esa_2020.addBands(landcover_remapped)

# Sample the image
# bands = ['B2', 'B3', 'B4', 'B8']
sample = training_data.select(bands + ['Map_remapped']).sample(
    region=aoi,
    scale=10,
    numPixels=5000,
    seed=2,
    geometries=True)

In [ ]:
# Add random column
sample = sample.randomColumn('random')

# Split
train = sample.filter(ee.Filter.lt('random', 0.7))
valid = sample.filter(ee.Filter.And(ee.Filter.gte('random', 0.7), ee.Filter.lt('random', 0.9)))
test = sample.filter(ee.Filter.gte('random', 0.9))

# Train the RF model
classifier = ee.Classifier.smileRandomForest(numberOfTrees=100).train(
    features=train,
    classProperty='Map_remapped',
    inputProperties=bands)

In [ ]:
# ESA_classes = landcover.get('Map_class_names').getInfo()
# ESA_palette = landcover.get('Map_class_palette').getInfo()
ESA_classes = [
    'Tree cover',
    'Shrubland',
    'Grassland',
    'Cropland',
    'Built-up',
    'Bare / sparse vegetation',
    'Snow and ice',
    'Permanent water bodies',
    'Herbaceous wetland',
    # 'Mangroves',
    'Moss and lichen'
]

# BASED IMAGE
ESA_palette = [
    '006400',
    'ffbb22',
    'ffff4c',
    'f096ff',
    'fa0000',
    'b4b4b4',
    'f0f0f0',
    '0064c8',
    '0096a0',
    # '00cf75',
    'fae6a0',
]

vis_params = {
    'min': 1,
    'max': 10,
    'palette': ESA_palette
}

thumb_params = {
    'dimensions': 512,
    'region': aoi,
    'format': 'png',
}


# Load and preprocess Sentinel-2 imagery for the given year
s2_2025 = s2.filterDate('2024-09-01', '2025-07-31').map(mask_s2_clouds).median().clip(aoi)
s2_2025_classified = s2_2025.select(bands).classify(classifier)
s2_2025_classified_rgb = s2_2025_classified.visualize(**vis_params)
s2_2025_url = s2_2025_classified_rgb.getThumbURL(thumb_params)

In [ ]:
s2_2020_classified = s2_esa_2020.select(bands).classify(classifier)
s2_2020_classified_rgb = s2_2020_classified.visualize(**vis_params)

s2_2018_classified = s2_2018.select(bands).classify(classifier)
s2_2018_classified_rgb = s2_2018_classified.visualize(**vis_params)

visualization = {
    'bands': ['Map'],
}

m = geemap.Map(center=[-36.1830, 175.3785], zoom=11)
m.addLayer(aoi, {}, 'AOI')
m.addLayer(s2_2025_classified_rgb, {}, 'S2 2025 Classified')
m.addLayer(s2_2020_classified_rgb, {}, 'S2 2020 Classified')
m.addLayer(s2_2018_classified_rgb, {}, 'S2 2018 Classified')
m.add_layer(landcover, visualization, 'Landcover')
m